## Take-Home Assignment#3 (Due: 11/06)
Data Science Bootcamp_gl2996

### LeetCode SQL & Pandas – Take-Home Assignment

- Each problem has **one SQL solution** and **one Pandas solution**.
- Comments are written in **English** and kept concise.
- Logic strictly follows each problem’s requirements without unnecessary complexity.


In [1]:
# Common imports for all Pandas solutions
import pandas as pd


### 1. 1050 – Actors and Directors Who Cooperated At Least Three Times

In [2]:
# SQL solution for LeetCode 1050
sql_1050 = """
SELECT actor_id, director_id
FROM ActorDirector
GROUP BY actor_id, director_id
HAVING COUNT(*) >= 3;
"""

# Pandas solution for LeetCode 1050
def actors_and_directors(actor_director: pd.DataFrame) -> pd.DataFrame:
    """Return (actor_id, director_id) pairs with at least 3 cooperations."""
    grouped = (
        actor_director
        .groupby(["actor_id", "director_id"])
        .size()
        .reset_index(name="cooperations")
    )
    return grouped[grouped["cooperations"] >= 3][["actor_id", "director_id"]]


### 2. 1667 – Fix Names in a Table

In [3]:
# SQL solution for LeetCode 1667
sql_1667 = """
SELECT 
    user_id,
    CONCAT(UPPER(SUBSTRING(name, 1, 1)),
           LOWER(SUBSTRING(name, 2))) AS name
FROM Users
ORDER BY user_id;
"""

# Pandas solution for LeetCode 1667
def fix_names(users: pd.DataFrame) -> pd.DataFrame:
    """Fix user names so only the first character is uppercase."""
    users = users.copy()
    users["name"] = users["name"].str.capitalize()
    return users.sort_values("user_id")[["user_id", "name"]]


### 3. 175 – Combine Two Tables

In [4]:
# SQL solution for LeetCode 175
sql_175 = """
SELECT 
    p.firstName,
    p.lastName,
    a.city,
    a.state
FROM Person p
LEFT JOIN Address a
  ON p.personId = a.personId;
"""

# Pandas solution for LeetCode 175
def combine_two_tables(person: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:
    """Left join Address onto Person by personId and select requested columns."""
    merged = person.merge(address, on="personId", how="left")
    return merged[["firstName", "lastName", "city", "state"]]


### 4. 176 – Second Highest Salary

In [5]:
# SQL solution for LeetCode 176
sql_176 = """
SELECT
    (
        SELECT DISTINCT salary
        FROM Employee
        ORDER BY salary DESC
        LIMIT 1 OFFSET 1
    ) AS SecondHighestSalary;
"""

# Pandas solution for LeetCode 176
def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    """Return the second highest distinct salary or None if it does not exist."""
    unique_salaries = (
        employee["salary"]
        .drop_duplicates()
        .sort_values(ascending=False)
    )
    if len(unique_salaries) < 2:
        value = None
    else:
        value = unique_salaries.iloc[1]
    return pd.DataFrame({"SecondHighestSalary": [value]})


### 5. 1327 – List the Products Ordered in a Period

In [6]:
# SQL solution for LeetCode 1327
sql_1327 = """
SELECT 
    p.product_name,
    SUM(o.unit) AS unit
FROM Products p
JOIN Orders o
  ON p.product_id = o.product_id
WHERE o.order_date BETWEEN '2020-02-01' AND '2020-02-29'
GROUP BY p.product_id, p.product_name
HAVING SUM(o.unit) >= 100;
"""

# Pandas solution for LeetCode 1327
def list_products(products: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    """Return products with at least 100 units ordered in February 2020."""
    orders = orders.copy()
    orders["order_date"] = pd.to_datetime(orders["order_date"])

    feb_orders = orders[
        (orders["order_date"] >= "2020-02-01") &
        (orders["order_date"] <= "2020-02-29")
    ]

    merged = feb_orders.merge(products, on="product_id", how="left")
    aggregated = (
        merged
        .groupby("product_name", as_index=False)["unit"]
        .sum()
    )
    return aggregated[aggregated["unit"] >= 100]


### 6. 1378 – Replace Employee ID With The Unique Identifier

In [7]:
# SQL solution for LeetCode 1378
sql_1378 = """
SELECT 
    u.unique_id,
    e.name
FROM Employees e
LEFT JOIN EmployeeUNI u
  ON e.id = u.id;
"""

# Pandas solution for LeetCode 1378
def replace_employee_id(employees: pd.DataFrame, employee_uni: pd.DataFrame) -> pd.DataFrame:
    """Left join EmployeeUNI onto Employees and return unique_id with name."""
    merged = employees.merge(employee_uni, on="id", how="left")
    return merged[["unique_id", "name"]]


### 7. 550 – Game Play Analysis IV

In [8]:
# SQL solution for LeetCode 550
sql_550 = """
SELECT 
    ROUND(
        COUNT(DISTINCT a2.player_id) / COUNT(DISTINCT a1.player_id),
        2
    ) AS fraction
FROM (
    SELECT player_id, MIN(event_date) AS first_login
    FROM Activity
    GROUP BY player_id
) a1
LEFT JOIN Activity a2
  ON a1.player_id = a2.player_id
 AND a2.event_date = DATE_ADD(a1.first_login, INTERVAL 1 DAY);
"""

# Pandas solution for LeetCode 550
def gameplay_analysis(activity: pd.DataFrame) -> pd.DataFrame:
    """Compute the fraction of players who logged in again the day after first login."""
    activity = activity.copy()
    activity["event_date"] = pd.to_datetime(activity["event_date"])

    first_login = (
        activity
        .groupby("player_id")["event_date"]
        .min()
        .reset_index()
    )
    first_login.columns = ["player_id", "first_login"]
    first_login["next_day"] = first_login["first_login"] + pd.Timedelta(days=1)

    merged = first_login.merge(
        activity,
        left_on=["player_id", "next_day"],
        right_on=["player_id", "event_date"],
        how="left"
    )

    total_players = len(first_login)
    logged_next_day = merged["event_date"].notna().sum()

    fraction = round(logged_next_day / total_players, 2) if total_players > 0 else 0.0
    return pd.DataFrame({"fraction": [fraction]})


### 8. 1075 – Project Employees I

In [9]:
# SQL solution for LeetCode 1075
sql_1075 = """
SELECT 
    p.project_id,
    ROUND(AVG(e.experience_years), 2) AS average_years
FROM Project p
JOIN Employee e
  ON p.employee_id = e.employee_id
GROUP BY p.project_id;
"""

# Pandas solution for LeetCode 1075
def project_employees_i(project: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    """Report average experience years of employees for each project."""
    merged = project.merge(employee, on="employee_id", how="left")
    avg = (
        merged
        .groupby("project_id", as_index=False)["experience_years"]
        .mean()
    )
    avg.rename(columns={"experience_years": "average_years"}, inplace=True)
    avg["average_years"] = avg["average_years"].round(2)
    return avg


### 9. 185 – Department Top Three Salaries

In [10]:
# SQL solution for LeetCode 185
sql_185 = """
WITH RankedSalaries AS (
    SELECT 
        d.name AS Department,
        e.name AS Employee,
        e.salary AS Salary,
        DENSE_RANK() OVER (
            PARTITION BY e.departmentId
            ORDER BY e.salary DESC
        ) AS rnk
    FROM Employee e
    JOIN Department d
      ON e.departmentId = d.id
)
SELECT Department, Employee, Salary
FROM RankedSalaries
WHERE rnk <= 3;
"""

# Pandas solution for LeetCode 185
def top_three_salaries(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    """Return employees whose salaries are in the top three distinct values per department."""
    merged = employee.merge(
        department,
        left_on="departmentId",
        right_on="id",
        suffixes=("_emp", "_dept")
    )

    merged["rnk"] = (
        merged
        .groupby("departmentId")["salary"]
        .rank(method="dense", ascending=False)
    )

    merged = merged[merged["rnk"] <= 3]

    result = merged[["name_dept", "name_emp", "salary"]].copy()
    result.columns = ["Department", "Employee", "Salary"]
    return result
